# Рубежный контроль №3
**"Информационное противоборство."**

**Выполнил: Александров А. Н., ИУ8-104**

**Вариант: 1**

## Задание.

1. Для 10 агентов случайным образом сгенерировать стохастическую матрицу доверия.
2. Получить результирующую матрицу доверия.
3. Случайным образом выбрать номера агентов из общего числа агентов для первого и второго игроков.
4. Определить функции выигрыша, целевые функции, точку утопии, найти аналитическое решение игры с не противоположными интересами двух игроков.

| Номер варианта | a | b | c | d | g_f | g_s |
|:--------------:|:-:|:-:|:-:|:-:|:---:|:---:|
|        1       | 1 | 2 | 1 | 5 |  1  |  2  |

In [39]:
import logging

import sympy
import numpy as np
# To represent multiple expressions in output of single cell.
from IPython.display import display

np.set_printoptions(precision=2, suppress=True)
logging.basicConfig(level=logging.INFO, format='%(message)s')

In [40]:
# Размерность матрицы (количество агентов)
N = 10
# Входные параметры.
a, b, c, d = 1, 2, 1, 5
g_f, g_s = 1, 2

### 1. Генерация стохастической матрицы
В модели социальной сети рассматриваются агенты ($i \in N = {1,2,..., n}, n \in \mathbb{N}$), составляющие эту сеть, и их
мнения $х_i$, зависящие от дискретного момента времени: $x_i = x_i(t), t = 0, 1, 2, ...$. 

В каждый момент времени мнения агентов изменяются под влиянием мнений других агентов. Степень этого влияния определяется неотрицательной стохастической по строкам матрицей $А = (a_{ij})$, где $a_{ij}$ — степень доверия $і$-го агента $j$-му агенту (в том числе и самому себе).

In [41]:
def generate_trust_matrix(n_agents):
    matrix = np.random.rand(n_agents, n_agents)
    # Нормализуем строки и столбцы, чтобы сумма значений в каждой строке была равна 1.
    return matrix / matrix.sum(axis=1, keepdims=True)


# Функция для проверки стохастичности матрицы
def is_stochastic(matrix):
    rows_sum = np.sum(matrix, axis=1)
    logging.info(f"Сумма по строкам: {rows_sum}")
    # Проверяем, что сумма значений в строках равна 1.
    return np.allclose(rows_sum, 1)


# Генерируем стохастическую матрицу доверия
trust_matrix = generate_trust_matrix(N)
trust_matrix

array([[0.13, 0.14, 0.1 , 0.01, 0.23, 0.12, 0.03, 0.05, 0.18, 0.  ],
       [0.1 , 0.  , 0.18, 0.09, 0.07, 0.1 , 0.08, 0.08, 0.09, 0.21],
       [0.07, 0.02, 0.03, 0.23, 0.08, 0.02, 0.23, 0.1 , 0.12, 0.11],
       [0.11, 0.03, 0.17, 0.05, 0.07, 0.07, 0.08, 0.08, 0.17, 0.17],
       [0.06, 0.24, 0.05, 0.01, 0.19, 0.15, 0.03, 0.09, 0.06, 0.13],
       [0.08, 0.11, 0.05, 0.04, 0.21, 0.03, 0.27, 0.15, 0.06, 0.01],
       [0.15, 0.15, 0.01, 0.18, 0.09, 0.  , 0.01, 0.2 , 0.05, 0.13],
       [0.16, 0.11, 0.16, 0.11, 0.06, 0.08, 0.14, 0.01, 0.04, 0.13],
       [0.09, 0.08, 0.05, 0.08, 0.01, 0.17, 0.  , 0.24, 0.17, 0.12],
       [0.13, 0.11, 0.1 , 0.06, 0.1 , 0.05, 0.1 , 0.11, 0.2 , 0.04]])

In [42]:
is_stochastic(trust_matrix)

Сумма по строкам: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


True

### 2. Мнение агентов

Вектор мнений $X = (x_1, x_2, ..., x_n)$ задан для начального момента времени ($X(0)$) и изменяется следующим образом:
$$x_i(t) = \sum_{j = 0}^{n} a_{ij}x_{j}(t - 1)$$

Этот закон можно записать иначе:
$$X(t) = A \cdot X(t ー 1)$$

При достаточно долгом взаимодействии агентов вектор мнений сходится к итоговому значению:
$$X = A^{\infty} \cdot X(0)$$
, где $A^{\infty} = \lim_{t \to \infty} A^{t}$.

In [43]:
def compute_final_opinions(trust_matrix, initial_opinions, epsilon=1e-6):
    i = 0
    logging.info(f"x({i}) = {initial_opinions}")
    current_opinions = initial_opinions.copy()
    previous_opinions = np.zeros_like(initial_opinions)
    res_trust_matrix = trust_matrix.copy()
    while np.linalg.norm(current_opinions - previous_opinions) > epsilon:
        previous_opinions = current_opinions
        current_opinions = np.dot(trust_matrix, current_opinions)
        res_trust_matrix = np.dot(res_trust_matrix, trust_matrix)
        i += 1
        logging.info(f"x({i}) = {current_opinions}")

    return current_opinions, res_trust_matrix

In [44]:
a, b = 1, 20
# Генерируем начальные мнения агентов.
initial_opinions = np.random.randint(a, b + 1, N)
final_opinions, res_trust_matrix = compute_final_opinions(trust_matrix, initial_opinions)
print(f"После взаимодействия агентов, вектор мнений сходится к значению:\nX = {final_opinions}")
print(f"Результирующая матрица доверия:\n{res_trust_matrix}")

x(0) = [ 7  8 13 20 12 13 20  1  7 11]
x(1) = [ 9.98 11.62 13.46 10.69  9.8  11.73  9.56 12.56  8.49 10.13]
x(2) = [10.59 10.9  10.31 10.72 10.94 10.55 10.86 10.84 10.98 10.58]
x(3) = [10.78 10.65 10.78 10.69 10.76 10.82 10.76 10.67 10.74 10.77]
x(4) = [10.75 10.76 10.73 10.75 10.74 10.74 10.71 10.75 10.73 10.74]
x(5) = [10.74 10.74 10.74 10.74 10.74 10.74 10.75 10.74 10.74 10.74]
x(6) = [10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74]
x(7) = [10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74]
x(8) = [10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74]
x(9) = [10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74]
x(10) = [10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74]
x(11) = [10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74]
x(12) = [10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74]
x(13) = [10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74]


После взаимодействия агентов, вектор мнений сходится к значению:
X = [10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74 10.74]
Результирующая матрица доверия:
[[0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]]


### 3. Информационное управление
Для информационного управления необходимо наличие нескольких игроков (в данном случае - двух), которые могут влиять на начальные мнения некоторых агентов. Случайным образом выберем число и номера агентов первого игрока и аналогично число и номера агентов второго игрока (они не должны пересекаться, так как эта ситуация выходит за рамки данной работы). Следует обратить внимание, что число агентов влияния первого игрока не обязательно должно совпадать с числом агентов влияния второго игрока.

In [45]:
def get_player_agents_indices(n_agents):
    # Генерируем случайное число агентов для каждого игрока (необязательно равное).
    player1_count = np.random.randint(1, n_agents)
    # Учитываем уже выбранных агентов для первого игрока.
    player2_count = np.random.randint(1, n_agents - player1_count + 1)

    # Генерируем случайные неповторяющиеся индексы агентов для каждого игрока
    player1_indices = np.random.choice(
        range(n_agents),
        size=player1_count, replace=False)
    player2_indices = np.random.choice(
        [idx for idx in range(n_agents) if idx not in player1_indices],
        size=player2_count, replace=False)

    player1_indices.sort()
    player2_indices.sort()
    return player1_indices, player2_indices

In [46]:
player1_influence_indices, player2_influence_indices = get_player_agents_indices(N)
print(f"Индексы агентов влияния для первого игрока: {player1_influence_indices}")
print(f"Индексы агентов влияния для второго игрока: {player2_influence_indices}")

Индексы агентов влияния для первого игрока: [0 2 3 4 5 7]
Индексы агентов влияния для второго игрока: [1 8 9]


Затем сгенерируем случайным образом управление для агентов влияния первого и второго игроков.
Сформируем вектор начальных мнений агентов с учетом информационного влияния, нейтральным агентам назначим случайные стартовые мнения. 

In [47]:
ABS_MAX_INFLUENCE = 100
DIFF = 20
player1_influence = np.random.randint(0, ABS_MAX_INFLUENCE)
print(f"Начальное мнение агентов 1 игрока: {player1_influence}")
player2_influence = np.random.randint(
    player1_influence - DIFF,
    player1_influence + DIFF,
)
print(f"Начальное мнение агентов 2 игрока: {player2_influence}")

Начальное мнение агентов 1 игрока: 80
Начальное мнение агентов 2 игрока: 87


In [48]:
# Модифицируем начальные мнения агентов с учётом влияния.
for i in player1_influence_indices:
    initial_opinions[i] = player1_influence

for i in player2_influence_indices:
    initial_opinions[i] = player2_influence

final_opinions, res_trust_matrix = compute_final_opinions(trust_matrix, initial_opinions)
print(f"После взаимодействия агентов, вектор мнений сходится к значению:\nX = {final_opinions}")
print(f"Результирующая матрица доверия:\n{res_trust_matrix}")

x(0) = [80 87 80 80 80 80 20 80 87 87]
x(1) = [80.19 77.13 67.78 77.78 81.44 65.28 81.65 73.54 82.46 76.18]
x(2) = [77.08 75.3  78.52 76.19 75.82 78.48 77.45 76.22 75.07 77.54]
x(3) = [76.45 77.19 76.63 76.93 76.58 76.55 76.33 77.07 76.66 76.48]
x(4) = [76.69 76.61 76.66 76.63 76.75 76.66 76.81 76.63 76.76 76.7 ]
x(5) = [76.7  76.69 76.71 76.7  76.68 76.71 76.66 76.69 76.67 76.7 ]
x(6) = [76.69 76.69 76.68 76.69 76.69 76.68 76.69 76.69 76.69 76.69]
x(7) = [76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69]
x(8) = [76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69]
x(9) = [76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69]
x(10) = [76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69]
x(11) = [76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69]
x(12) = [76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69]
x(13) = [76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69]
x(14) = [76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.

После взаимодействия агентов, вектор мнений сходится к значению:
X = [76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69 76.69]
Результирующая матрица доверия:
[[0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]
 [0.11 0.1  0.09 0.08 0.11 0.08 0.09 0.11 0.11 0.11]]


### 4. Определение победителя

In [49]:
r_f = sum(res_trust_matrix[0][player1_influence_indices])
r_s = sum(res_trust_matrix[1][player2_influence_indices])
r_f, r_s

(0.5852131662273325, 0.3220461589765842)

In [50]:
u, v = sympy.symbols("u v")
# Задаём целевые функции обоих игроков.
X = u * r_f + v * r_s
target_f = a * X - b * X ** 2 - g_f * u ** 2 / 2
target_s = c * X - d * X ** 2 - g_s * v ** 2 / 2
print(f"Целевые функции игроков:\n"
      f"Ф_f(u, v) = {sympy.N(target_f, 3)}\n"
      f"Ф_s(u, v) = {sympy.N(target_s, 3)}")

Целевые функции игроков:
Ф_f(u, v) = -0.5*u**2 + 0.585*u + 0.322*v - 6.85*(u + 0.55*v)**2
Ф_s(u, v) = 0.585*u - v**2 + 0.322*v - 1.71*(u + 0.55*v)**2


In [51]:
target_f_diff = target_f.diff(u)
target_s_diff = target_f.diff(v)
print(f"Частные производные целевые функций:\n"
      f"Ф_f(u, v)|_u = {sympy.N(target_f_diff, 3)}\n"
      f"Ф_s(u, v)|_v = {sympy.N(target_s_diff, 3)}")

Частные производные целевые функций:
Ф_f(u, v)|_u = -14.7*u - 7.54*v + 0.585
Ф_s(u, v)|_v = -7.54*u - 4.15*v + 0.322


In [52]:
# Находим значения начальных мнений агентов `u` и `v`.
solution = sympy.solve(
    (
        sympy.Eq(target_f.diff(u), 0),
        sympy.Eq(target_s.diff(v), 0)
    ),
    (u, v),
)
u, v = solution[u], solution[v]
print(f"Решая систему, получим: u = {u:.3f}, v = {v:.3f}")

Решая систему, получим: u = -0.021, v = 0.119


In [53]:
X = float(u * r_f + v * r_s)
print(f"Найдём точку утопии X = {X:.3f}")

Найдём точку утопии X = 0.026


In [54]:
x_max_f, x_max_s = a / (2 * b), c / (2 * d)
delta_x_f = abs(X - x_max_f)
delta_x_s = abs(X - x_max_s)
print(f"При X_max_f ={x_max_f}, X_max_s= {x_max_s} найдем расстояния:\n"
      f"𝛥𝑥_𝑓 = {delta_x_f:.3f}\n"
      f"𝛥𝑥_𝑠 = {delta_x_s:.3f}")
if delta_x_f < delta_x_s:
    print("Как мы видим 𝛥𝑥_𝑓 < 𝛥𝑥_𝑠, поэтому выиграл первый игрок.")
else:
    print("Как мы видим 𝛥𝑥_𝑓 > 𝛥𝑥_𝑠, поэтому выиграл второй игрок.")

При X_max_f =0.025, X_max_s= 0.1 найдем расстояния:
𝛥𝑥_𝑓 = 0.001
𝛥𝑥_𝑠 = 0.074
Как мы видим 𝛥𝑥_𝑓 < 𝛥𝑥_𝑠, поэтому выиграл первый игрок.
